# ISCE3 Sentinel-1 RTC

Alex Lewandowski; Alaska Satellite Facility

In [ ]:
import asf_search
import boto3
from botocore import UNSIGNED
from botocore.config import Config
from bs4 import BeautifulSoup
from datetime import datetime
import getpass
import numpy as np
from osgeo import gdal
from pathlib import Path
import re
import requests
import rtc
import shutil
from urllib.parse import urljoin
import yaml
from zipfile import ZipFile

## Authenticate with asf_search and download and extract a Sentinel-1 scene

In [ ]:
username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf_search.ASFSession().auth_with_creds(username, password)
except asf_search.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

In [ ]:
# S1A_IW_SLC__1SDV_20230108T135223_20230108T135251_046693_0598D3_A89F +-
# S1A_IW_SLC__1SDV_20230109T012613_20230109T012640_046700_05990D_10DF ++
# S1A_IW_SLC__1SDV_20230112T235553_20230112T235623_046757_059B03_FABC --
# S1A_IW_SLC__1SDV_20230108T213922_20230108T213949_046698_0598FF_D885 -+

scene = input("Enter a Sentinel-1 Scene name")

In [ ]:
# TODO create dir named for s1-scene and adjust all upstream paths to download data to and save output there 

data_dir_path = Path.cwd()/scene
data_dir_path.mkdir(exist_ok=True)

In [ ]:
zip_path = data_dir_path/f"{scene}.zip"

if not zip_path.exists():
    results = asf_search.granule_search(scene)
    results[0].download(data_dir_path, session=user_pass_session)

with ZipFile(zip_path, 'r') as z:
    z.extractall(data_dir_path)

## Download the Sentinel-1 orbit file

In [ ]:
def datetime_in_orbitfile_range(datetime_str, orbitfile_name):
    orbit_date_regex = "(?<=\w[0-9]{7}T[0-9]{6}_V)\w[0-9]{7}T[0-9]{6}_\w[0-9]{7}T[0-9]{6}(?=\.EOF)"
    orbit_dates = re.search(orbit_date_regex, orbitfile_name)
    if orbit_dates:
        orbit_dates = orbit_dates.group(0).split('_')
    else:
        return False
    return orbit_dates[0] < datetime_str < orbit_dates[1]

def write_dot_netrc(path, username, password):
    with open(netrc_path, 'w+') as netrc:
        netrc.write(f'machine urs.earthdata.nasa.gov login {username} password {password}')

In [ ]:
if scene[:3] == "S1A":
    platform = "S1A"
elif scene[:3] == "S1B":
    platform = "S1B"
else:
    raise Exception("Platform not found in scene name")

s1_date_regex = "\w[0-9]{7}T[0-9]{6}"
datetime_str = re.search(s1_date_regex, scene).group(0)

In [ ]:
orbit_index_url = "https://s1qc.asf.alaska.edu/aux_poeorb/"
soup = BeautifulSoup(requests.get(orbit_index_url).content, "html.parser")

orbitfile_links = list(soup.select('a'))
orbitfile_link = [l for l in orbitfile_links if platform in str(l) and datetime_in_orbitfile_range(datetime_str, str(l))][0]
orbitfile_name = orbitfile_link.attrs.get("href")

href = urljoin(orbit_index_url, orbitfile_name)

In [ ]:
netrc_path = Path('/home/jovyan/.netrc')
write_dot_netrc(netrc_path, username, password)

orbitfile_path = data_dir_path/orbitfile_name
r = requests.get(href)
with open(orbitfile_path, 'wb') as f:
    f.write(r.content)

netrc_path.unlink()

## Download the COP30 DEM

In [ ]:
# TODO access bounds in manifest.safe without extracting entire zip, or maybe with asf_search?
# TODO: test for positive and negative latitudes and longitudes
safe_path = data_dir_path/f"{scene}.SAFE"
with open(safe_path/"manifest.safe", 'r') as f:
    data = f.read()
bs_data = BeautifulSoup(data, "xml")
extents = bs_data.find_all('gml:coordinates')
extents = extents[0].string.split(' ')
extents = [[int(np.floor(float(x.split(',')[0]))), int(np.floor(float(x.split(',')[1])))] for x in extents]

lats = list(set([c[0] for c in extents]))
lats = list(range(min(lats), max(lats)+1))    
longs = list(set([c[1] for c in extents]))
longs = list(range(min(longs), max(longs)+1))

In [ ]:
dem_s3_paths = []

for lat in lats:
    for long in longs:
        if lat >= 0:
            lat_dir = "N"
        else:
            lat_dir = "S"
        if long >= 0:
            long_dir = "E"
        else:
            long_dir = "W"
        
        dem_s3_paths.append(f"Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{long_dir}{int(abs(long)):03d}_00_DEM/Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{long_dir}{int(abs(long)):03d}_00_DEM.tif")
dem_s3_paths

In [ ]:
dem_dir_path = data_dir_path/"dems"
dem_dir_path.mkdir(exist_ok=True)

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))

bucket_name = "copernicus-dem-30m"
bucket = s3.Bucket(bucket_name)

for s3_path in dem_s3_paths:
    bucket.download_file(s3_path, dem_dir_path/s3_path.split('/')[1]) 

In [ ]:
dem_paths = ' '.join([str(p) for p in (dem_dir_path.glob("*DEM.tif"))])
dem_paths

In [ ]:
merged_dem_path = dem_dir_path/"merged_dem.tif"
!gdal_merge.py -n 0.0 -o $merged_dem_path $dem_paths

## Copy and edit a template rtc_s1.yaml 

This is the config file we will use to define parameters for RTC processing

In [ ]:
rtc_config_template_path = Path(rtc.__file__).parent/"defaults/rtc_s1.yaml"
rtc_config_path = data_dir_path/"rtc_s1.yaml"
shutil.copyfile(rtc_config_template_path, rtc_config_path)

In [ ]:
with open(rtc_config_path, 'r') as f:
    config = yaml.safe_load(f)

In [ ]:
config

In [ ]:
config['runconfig']['groups']['input_file_group']['safe_file_path'] = [str(data_dir_path/f"{scene}.zip")]
config['runconfig']['groups']['input_file_group']['orbit_file_path'] = [str(orbitfile_path)]
config['runconfig']['groups']['dynamic_ancillary_file_group']['dem_file'] = str(merged_dem_path)
config['runconfig']['groups']['dynamic_ancillary_file_group']['dem_description'] = f"Merged COP30 DEMS from s3://copernicus-dem-30m: {' '.join([p.split('/')[1] for p in dem_s3_paths])}"
config['runconfig']['groups']['product_group']['output_dir'] = str(data_dir_path/'ISCE3_RTC')
config['runconfig']['groups']['product_group']['save_mosaics'] = True
config['runconfig']['groups']['product_group']['product_path'] = '.'
config['runconfig']['groups']['product_group']['scratch_path'] = str(data_dir_path/'scratch_dir')
config['runconfig']['groups']['product_group']['product_id'] = f'OPERA_L2_RTC-S1_{scene}'
config['runconfig']['groups']['processing']['polarization'] = 'dual-pol'

In [ ]:
with open(rtc_config_path, 'w') as f:
    yaml.dump(config, f)

## Run rtc_s1_single_job.py

- Open rtc_s1.yaml and adjust any parameters you like

In [ ]:
!rtc_s1_single_job.py $rtc_config_path